In [1]:

##############################################
###    this cell is for dataset setting    ###   
##############################################

import os
import PIL
import torch
import torchvision

import thirdparty.torchvision.detection.utils
import utilities.datasets.label_me_dataset

dataset_root = './data/dataset'

#---setting picture size, please set as same as your data---

resized_width = 500
resized_height = 600

#---setting data form---

form = 'bmp'

#-----------------------
dataset_train_image_root = os.path.join(dataset_root, 'image')
dataset_train_mask_root  = os.path.join(dataset_root, 'mask')
dataset_val_image_root   = os.path.join(dataset_root, 'val_image')
dataset_val_mask_root    = os.path.join(dataset_root, 'val_mask')
dataset_test_image_root  = os.path.join(dataset_root, 'test')
resized_size = (resized_height, resized_width)
transforms = {}

#---data transform function, you can set any function you like inside, please refer to pytorch document----

transforms['images'] = torchvision.transforms.Compose([torchvision.transforms.Resize(resized_size),
                                                       torchvision.transforms.ToTensor(),])

###########################################################################################################

transforms['masks'] = torchvision.transforms.Resize(resized_size, PIL.Image.NEAREST)

dataset = {}
dataset['train'] = utilities.datasets.label_me_dataset.LabelMeDataset(dataset_train_image_root,
                                                                      dataset_train_mask_root,
                                                                      form,
                                                                      transforms['images'],
                                                                      transforms['masks'])
dataset['val']   = utilities.datasets.label_me_dataset.LabelMeDataset(dataset_val_image_root,
                                                                      dataset_val_mask_root,
                                                                      form,
                                                                      transforms['images'],
                                                                      transforms['masks'])
dataset['test']  = utilities.datasets.label_me_dataset.LabelMeDataset(dataset_test_image_root,
                                                                      None,
                                                                      form,
                                                                      transforms['images'],
                                                                      None)
data_loader = {}
data_loader['train'] = torch.utils.data.DataLoader(dataset['train'],
                                                   batch_size=1,
                                                   shuffle=True,
                                                   num_workers=1,
                                                   collate_fn=thirdparty.torchvision.detection.utils.collate_fn)
data_loader['val'] = torch.utils.data.DataLoader(dataset['val'],
                                                 batch_size=1,
                                                 shuffle=False,
                                                 num_workers=1, 
                                                 collate_fn=thirdparty.torchvision.detection.utils.collate_fn)

print('train:' + str(len(data_loader['train'])))
print('Val:' + str(len(data_loader['val'])))

The dataset is for testing only.
train:266
Val:88


In [2]:

#######################################################
###    this cell is to training function setting    ###   
#######################################################

import utilities.engine
import utilities.models.mask_rcnn
import thirdparty.torchvision.detection.engine

#---setting training parameters, we only provide mask-RCNN with Resnet50 backbone model now
#---for other training function, please refer to Pytorch document--------------------------

num_classes = 2
model       = utilities.models.mask_rcnn.MaskRCNN(number_classes = num_classes)
criterion   = None
optimizer   = torch.optim.SGD(params = [p for p in model.parameters() if p.requires_grad],
                              lr = 0.005,
                              momentum=0.9,
                              weight_decay=0.0005) 
lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
weights     = "./weights/TSMC_jig_detection.pth" 
to_load     = True
num_epochs  = 10

###########################################################################################

engine = utilities.engine.Engine(model = model, criterion = criterion, optimizer = optimizer, device = True)
# if to_load and os.path.isfile(weights):
#     engine.load(weights)
# for epoch in range(num_epochs):
#     thirdparty.torchvision.detection.engine.train_one_epoch(engine._model,
#                                                             engine._optimizer,
#                                                             data_loader['train'],
#                                                             engine._device,
#                                                             epoch,
#                                                             print_freq=10)
#     lr_schedule.step()
#     thirdparty.torchvision.detection.engine.evaluate(engine._model, data_loader['val'], device=engine._device)
#     engine.save(weights)
print("That's it!")

That's it!


In [3]:

######################################################
###    this cell is to visualize testing result    ###   
######################################################

import cv2
import matplotlib.pyplot as plt
import numpy as np
import utilities.engine
import utilities.tools.general
import utilities.tools.instance_segmentation
# np.set_printoptions(threshold=np.inf)

#---put in your trained weight and class name file, then set the shreshold---

weights     = './weights/TSMC_jig_detection.pth'
class_names = './data/TSMC_Jig_detection.names.txt'
threshold   = 0.8  #range:0~0.9, higher means higher score prediction can pass

##############################################################################

engine.load(weights)
image, target = utilities.tools.general.get_random_data(dataset['test'])
masks, boxes, labels = utilities.tools.instance_segmentation.get_predictions(engine,
                                                                             utilities.tools.general.get_classes(class_names),
                                                                             image,
                                                                             is_tensor = True,
                                                                             threshold = threshold)

masks_list = utilities.tools.instance_segmentation.get_masks_polygon(masks)


utilities.tools.general.print_predictions_number(len(labels))
image = image.mul(255).permute(1, 2, 0).byte().numpy()
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
result = utilities.tools.instance_segmentation.get_masked_image(image, (masks, boxes, labels))

plt.figure(figsize=(20,30))
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.xticks([])
plt.yticks([])
plt.show()

TypeError: get_outputs() takes 2 positional arguments but 3 were given

In [ ]:
target = str(target).lstrip("{'image_path'): '").rstrip(".bmp'}")
with open(str(target), 'w+') as txt_file:
    txt_file.write(str(masks_list))
    txt_file.write('\n')
    txt_file.write('lets go')

In [ ]:
# import numpy as np
# import pandas as pd

# masks = masks.squeeze()
# data_df = pd.DataFrame(masks)

# # change the index and column name
# data_df.columns = list(range(1,501))
# data_df.index = list(range(1,601))


# # create and writer pd.DataFrame to excel
# writer = pd.ExcelWriter('./masks.xlsx')
# data_df.to_excel(writer,'page_1',float_format='%.5f') # float_format 控制精度
# writer.save()